In [7]:
#!pip uninstall onnx==1.8.1 -y
#!pip uninstall mxnet -y
#!pip uninstall numpy -y

In [1]:
#!pip install 'h5py==2.10.0' --force-reinstall
#!pip install numpy
#!pip install onnx==1.8.1
#!pip install mxnet
#!pip install pytorch2keras
#!pip install caffe2

In [2]:
from __future__ import print_function, division
#Import needed packages
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#from onnx_tf.backend import prepare
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.autograd import Variable
from pytorch2keras.converter import pytorch_to_keras
import matplotlib.pyplot as plt
import time
import os
import copy
import csv
from tensorflow import keras
import tensorflow as tf
import pandas as pd
from datetime import datetime
from sklearn import metrics

plt.ion()   # interactive mode

In [2]:
from numba import cuda 
device_reset = cuda.get_current_device()
device_reset.reset()

/usagers3/opmos/anaconda3/lib/python3.8/site-packages/numba/core/errors.py:154: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


In [3]:
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [4]:
# used for Alexnet
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 250
# percentage of training set to use as validation
valid_size = 0.2
transform_train = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# choose the training and test datasets
train_data = datasets.CIFAR10('data', train=True,
                              download=True, transform=transform_train)
test_data = datasets.CIFAR10('data', train=False,
                             download=True, transform=transform_test)

# prepare data loaders (combine dataset and sampler)
trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
     num_workers=num_workers)
#valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
#    sampler=valid_sampler, num_workers=num_workers)
valloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# specify the image classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

Files already downloaded and verified
Files already downloaded and verified


In [5]:
# convert data to a normalized torch.FloatTensor
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
# choose the training and test datasets
train_data = datasets.CIFAR10('data', train=True,
                              download=True, transform=transform)
test_data = datasets.CIFAR10('data', train=False,
                             download=True, transform=transform)

# obtain training indices that will be used for validation
# define samplers for obtaining training and validation batches
#train_sampler = SubsetRandomSampler(train_idx)
#valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders (combine dataset and sampler)
trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
     num_workers=num_workers)
#valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
#    sampler=valid_sampler, num_workers=num_workers)
valloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# specify the image classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

Files already downloaded and verified
Files already downloaded and verified


In [6]:
transform = transforms.Compose([
    transforms.Resize(size=(32, 32)),
    transforms.ToTensor(),
    transforms.Normalize( 
       (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010) 
    )
])
train_dataset = torchvision.datasets.CIFAR10(
    root= './data', train = True,
    download =True, transform = transform)
test_dataset = torchvision.datasets.CIFAR10(
    root= './data', train = False,
    download =True, transform = transform)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
# Mnist dataset fpr Lenet
batch_size = 500
T = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])
train_data = torchvision.datasets.MNIST('mnist_data', train=True, download=True, transform=T)
val_data = torchvision.datasets.MNIST('mnist_data', train=False, download=True, transform=T)

trainloader = torch.utils.data.DataLoader(train_data, batch_size = batch_size)
valloader = torch.utils.data.DataLoader(val_data, batch_size = batch_size)

/usagers3/opmos/anaconda3/lib/python3.8/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [6]:
def predict_dl(model, data):
    y_pred = []
    y_true = []
    for i, (images, labels) in enumerate(data):
        images = images.cuda()
        x = model(images)
        value, pred = torch.max(x, 1)
        pred = pred.data.cpu()
        y_pred.extend(list(pred.numpy()))
        y_true.extend(list(labels.numpy()))
    return np.array(y_pred), np.array(y_true)

In [7]:
def top_n_predict_dl(model, data, n=10):
    pred_dict = {}
    for i in range (n):
        y_pred, y_true = predict_dl(resnet50_model, valloader)
        print('predict: {}/{}'.format(i, n))
        for x in range (len(y_pred)):
            if y_true[x] in pred_dict.keys():
                val = pred_dict.get(y_true[x])
                if len(val) < n: 
                    val.append(y_pred[x])
                pred_dict[y_true[x]] = val
            else:
                pred_dict[y_true[x]] = [y_pred[x]]
    y_true = []
    y_scores = []
    for key, val in pred_dict.items():
        y_true.append(key)
        y_scores.append(val)
    return pred_dict, y_true, y_scores

In [8]:
def validate(model, data):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(data):
        images = images.cuda()
        x = model(images)
        value, pred = torch.max(x,1)
        pred = pred.data.cpu()
        total += x.size(0)
        correct += torch.sum(pred == labels)
    return correct*100./total

In [8]:
path = './data/19-08-2021/'

In [10]:
# Model class must be defined somewhere
since_0 = time.time()
#model_name = 'torch_resnet50-cifar10_2021-08-19-15:05:17'
#model_name = 'torch_resnet18_cifar10_2021-08-20-15:17:40'
model_name = 'torch_vgg_cifar10_2021-08-23-17:19:32'
#model_name = 'torch_alexnet_cifar10_2021-08-20-15:26:26'
#model_name = 'torch_alexnet_cifar10_2021-08-20-15:26:26'
#model_name = 'torch_Lenet5-mnist_2021-09-03-13:54:37'
model = torch.load(path+model_name+'.pth')
#resnet50_model.eval()
t_elapsed_0 = time.time() - since_0
size0 = os.path.getsize(path+model_name+'.pth')
size0

515298923

In [11]:
size1 = os.path.getsize(path+model_name+'.pth')
size1

515298923

In [13]:
y_pred, y_true = predict_dl(model, valloader)

In [44]:
#pred_dict, y_true_, y_scores = top_n_predict_dl(resnet50_model, valloader, 10)

In [45]:
#y_scores

In [46]:
#top5_scores = metrics.top_k_accuracy_score(y_pred, y_scores, k=10)
#print('Top5 score: %f' % top5_scores)

#top5_scores_normalised = metrics.top_k_accuracy_score(y_pred, y_scores, k=5, normalize=False)
#print('Top5 score normalised: %f' % top5_scores_normalised)

In [47]:
# accuracy: (tp + tn) / (p + n)
accuracy = metrics.accuracy_score(y_true, y_pred)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = metrics.precision_score(y_true, y_pred, average='macro')
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = metrics.recall_score(y_true, y_pred, average='macro')
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = metrics.f1_score(y_true, y_pred, average='macro')
print('F1 score: %f' % f1)

Accuracy: 0.523300
Precision: 0.522173
Recall: 0.523300
F1 score: 0.521615


In [14]:
scores = validate(resnet50_model, valloader)

In [15]:
scores

tensor(52.3300)

## Lets export the model to onnx format

In [20]:
#input_np = np.random.uniform(0, 1, (1, 3, 224, 224))
#input_var = Variable(torch.FloatTensor(input_np))
#k_resnet50_model = pytorch_to_keras(resnet50_model, input_var.cuda(), [(3, 224, 224,)], verbose=True, change_ordering=True)

In [23]:
!pip uninstall onnxruntime -y
!pip uninstall onnxruntime-gpu -y
!pip install onnxruntime

Found existing installation: onnxruntime-gpu 1.9.0
Uninstalling onnxruntime-gpu-1.9.0:
  Successfully uninstalled onnxruntime-gpu-1.9.0
     |████████████████████████████████| 4.8 MB 11.3 MB/s eta 0:00:01


In [12]:
import onnx
import onnxruntime
import time

In [13]:
import caffe2.python.onnx.backend as onnx_caffe2_backend

In [20]:
def to_onnx(i, x, abs_errors,rel_errors, t0_list, t1_list, t2_list, t3_list, s_list):
    # Input to the model
    #device_reset = cuda.get_current_device()
    #device_reset.reset()
    #x.cuda()
   
    print("converting for batch: ", i)
    
    torch.random.manual_seed(42)
    #x = torch.randn(10000, 3, 32, 32, requires_grad=True)
    since_1 = time.time()
    #model = torch.load(path+model_name+'.pth')
    torch_out = model(x)
    t_elapsed_1 = time.time() - since_1
    # Export the model
    since_1 = time.time()
    torch.onnx._export(model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "./data/ONNX/torch/model_ft-{}.onnx".format(model_name),   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  #operator_export_type=torch.onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK,
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})
    t_elapsed_2 = time.time() - since_1
    
    
    onnx_model = onnx.load("./data/ONNX/torch/model_ft-{}.onnx".format(model_name))
    onnx.checker.check_model(onnx_model)
    
    size2 = os.path.getsize("./data/ONNX/torch/model_ft-{}.onnx".format(model_name))
    s_list.append(size2)
    
    def to_numpy(tensor):
        return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
    try:
        ort_session = onnxruntime.InferenceSession("./data/ONNX/torch/model_ft-{}.onnx".format(model_name))
        since_1 = time.time()
        ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
        ort_outs = ort_session.run(None, ort_inputs)
    except:
        prepared_backend = onnx_caffe2_backend.prepare(onnx_model)
        since_1 = time.time()
        W = {model.graph.input[0].name: to_numpy(x)}
        ort_outs = prepared_backend.run(W)
        
   
    t_elapsed_3 = time.time() - since_1
    # compare ONNX Runtime and PyTorch results
    print("\n*********\n\n")
    #time_diff = t_elapsed_0+t_elapsed_1, t_elapsed_2, t_elapsed_3
    t0_list.append(t_elapsed_0)
    t1_list.append(t_elapsed_1)
    t2_list.append(t_elapsed_2)
    t3_list.append(t_elapsed_3)
    
    abs_err = np.absolute(to_numpy(torch_out)-ort_outs[0])
    rel_err = np.absolute(to_numpy(torch_out)-ort_outs[0])/ np.absolute(ort_outs[0])
    abs_errors.append(abs_err)
    rel_errors.append(rel_err)
    
    return (abs_err, rel_err)

In [21]:
def _lets_convert(data):
    since = time.time()
    list_converted = []
    t0_list = []
    t1_list = []
    t2_list = []
    t3_list = []
    s_list = []
    abs_errors = []
    rel_errors = []
    for i, (images, labels) in enumerate(data):
        torch.cuda.empty_cache()
        images = images.cuda()
        list_converted.append(to_onnx(i, images, abs_errors,rel_errors, t0_list, t1_list, t2_list, t3_list, s_list))
        if i == 15:
            break
    time_elapsed = time.time() - since
    print('Conversion complete in {:.0f}m {:.0f}s,  Loading Pytorch: {}, Pytorch time: {:.4f}, conversion time: {:.4f}, onnx runtime: {:.4f}'.format(
        time_elapsed // 60, time_elapsed % 60, np.mean(np.array(t0_list)), np.mean(np.array(t1_list)), np.mean(np.array(t2_list)), np.mean(np.array(t3_list))) )
    
    #return list_converted
    return list_converted, abs_errors, rel_errors, 'Conversion complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60), np.mean(np.array(t0_list)), np.mean(np.array(t1_list)), np.mean(np.array(t2_list)), np.mean(np.array(t3_list)), np.mean(np.array(s_list))

In [22]:
#list_converted = _lets_convert(valloader)
#list_converted = _lets_convert(test_ds)
model_name = 'VGG-torch'
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
import pandas as pd 
for batch_size in [1, 5,10,20,30,40,50,60,70,80,90,100,128, 150, 200, 250,300,350, 400, 450, 500]:
    print("################ Batch size: ", batch_size)
    # prepare data loaders (combine dataset and sampler)
    trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
     num_workers=num_workers)
    #valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
    #    sampler=valid_sampler, num_workers=num_workers)
    valloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)
    
    # specify the image classes
    classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

    list_converted, abs_errors, rel_errors, total_time, t0, t1, t2, t3, file_size = _lets_convert(valloader)
    for i in range(len(abs_errors)):
        if i == 0:
            abs_array = abs_errors[i]
            rel_array = rel_errors[i]
        else:
            np.append(abs_array, abs_errors[i])
            np.append(rel_array, rel_errors[i])
            
    abs_list = []
    rel_list = []
    model_list = []
    batch_list = []
    summary_list = ['Modelsize:{}, Conversion: {}, Loading: {}, t1: {}, conversion time: {}, onnx runtime: {}, onnx filesize: {}'.format(size0, total_time, t0, t1, t2, t3, file_size)]
    for i in range(len(abs_array)):
        abs_list.append(abs_array[i][0])
        rel = rel_array[i][0]
        if rel == np.inf or rel == -np.inf:
            rel = 0.0
        rel_list.append(rel)
        batch_list.append(batch_size)
        model_list.append(model_name)
        if i >= len(summary_list):
            summary_list.append('')
    print(len(summary_list), len(rel_list))
    data = pd.DataFrame({'model':model_list,'batch_size': batch_list, 'abs_errors':abs_list, 'rel_errors':rel_list, 'summary': summary_list})
    data.to_csv('./data/errors/torch2/tf_errors_{}_{}.csv'.format(model_name, batch_size))

################ Batch size:  1
converting for batch:  0


/usagers3/opmos/anaconda3/lib/python3.8/site-packages/caffe2/python/onnx/backend.py:688: UserWarning: This version of onnx-caffe2 targets ONNX operator set version 9, but the model we are trying to import uses version 10.  We will try to import it anyway, but if the model uses operators which had BC-breaking changes in the intervening versions, import will fail.
  warnings.warn("This version of onnx-caffe2 targets ONNX operator set version {}, but the model we are trying to import uses version {}.  We will try to import it anyway, but if the model uses operators which had BC-breaking changes in the intervening versions, import will fail.".format(cls._known_opset_version, imp.version))
/usagers3/opmos/anaconda3/lib/python3.8/site-packages/caffe2/python/onnx/backend.py:690: UserWarning: Unrecognized operator set org.pytorch.aten
  warnings.warn("Unrecognized operator set {}".format(imp.domain))
ONNX FATAL: Error while processing node: input: "91"
input: "92"
output: "93"
op_type: "adapti

RuntimeError: ONNX conversion failed, encountered 1 errors:

Error while processing node: input: "91"
input: "92"
output: "93"
op_type: "adaptive_avg_pool2d"
domain: "org.pytorch.aten"
. Exception: Don't know how to translate op adaptive_avg_pool2d

In [19]:
#list_converted = _lets_convert(valloader)
#list_converted = _lets_convert(test_ds)
model_name = 'lenet5-torch'
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
import pandas as pd 
for batch_size in [1, 5,10,20,30,40,50,60,70,80,90,100,28, 150,200, 250,300,350, 400, 450, 500]:
    print("################ Batch size: ", batch_size)
    trainloader = torch.utils.data.DataLoader(train_data, batch_size = batch_size)
    valloader = torch.utils.data.DataLoader(val_data, batch_size = batch_size)
    
    # specify the image classes
    classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

    list_converted, abs_errors, rel_errors, total_time, t0, t1, t2, t3, file_size = _lets_convert(valloader)
    for i in range(len(abs_errors)):
        if i == 0:
            abs_array = abs_errors[i]
            rel_array = rel_errors[i]
        else:
            np.append(abs_array, abs_errors[i])
            np.append(rel_array, rel_errors[i])
            
    abs_list = []
    rel_list = []
    model_list = []
    batch_list = []
    summary_list = ['Modelsize:{}, Conversion: {}, Loading: {}, t1: {}, conversion time: {}, onnx runtime: {}, onnx filesize: {}'.format(size0, total_time, t0, t1, t2, t3, file_size)]
    for i in range(len(abs_array)):
        abs_list.append(abs_array[i][0])
        rel = rel_array[i][0]
        if rel == np.inf or rel == -np.inf:
            rel = 0.0
        rel_list.append(rel)
        batch_list.append(batch_size)
        model_list.append(model_name)
        if i >= len(summary_list):
            summary_list.append('')
    print(len(summary_list), len(rel_list))
    data = pd.DataFrame({'model':model_list,'batch_size': batch_list, 'abs_errors':abs_list, 'rel_errors':rel_list, 'summary': summary_list})
    data.to_csv('./data/errors/torch2/tf_errors_{}_{}.csv'.format(model_name, batch_size))

################ Batch size:  1
converting for batch:  0

*********


converting for batch:  1

*********


converting for batch:  2

*********


converting for batch:  3

*********


converting for batch:  4

*********


converting for batch:  5

*********


converting for batch:  6

*********


converting for batch:  7

*********


converting for batch:  8

*********


converting for batch:  9

*********


converting for batch:  10

*********


Conversion complete in 0m 1s,  Loading Pytorch: 0.00800633430480957, Pytorch time: 0.0024, conversion time: 0.0966, onnx runtime: 0.0007
1 1
################ Batch size:  5
converting for batch:  0

*********


converting for batch:  1

*********


converting for batch:  2

*********


converting for batch:  3

*********


converting for batch:  4

*********


converting for batch:  5

*********


converting for batch:  6

*********


converting for batch:  7

*********


converting for batch:  8

*********


converting for batch:  9

********


*********


converting for batch:  1

*********


converting for batch:  2

*********


converting for batch:  3

*********


converting for batch:  4

*********


converting for batch:  5

*********


converting for batch:  6

*********


converting for batch:  7

*********


converting for batch:  8

*********


converting for batch:  9

*********


converting for batch:  10

*********


Conversion complete in 0m 2s,  Loading Pytorch: 0.00800633430480957, Pytorch time: 0.0018, conversion time: 0.0893, onnx runtime: 0.0093
250 250
################ Batch size:  300
converting for batch:  0

*********


converting for batch:  1

*********


converting for batch:  2

*********


converting for batch:  3

*********


converting for batch:  4

*********


converting for batch:  5

*********


converting for batch:  6

*********


converting for batch:  7

*********


converting for batch:  8

*********


converting for batch:  9

*********


converting for batch:  10

*********


Conversi

In [17]:
list_converted = _lets_convert(valloader)
#list_converted = _lets_convert(test_ds)
model_name = 'VGG-torch'
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
import pandas as pd 
for batch_size in [1, 5,10,20,30,40,50,60,70,80,90,100,28, 150,200, 250,300,350, 400, 450, 500]:
    print("################ Batch size: ", batch_size)
    #trainloader = torch.utils.data.DataLoader(train_data, batch_size = batch_size)
    #valloader = torch.utils.data.DataLoader(val_data, batch_size = batch_size)
    
    #train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, shuffle = True)
    
    # specify the image classes
    classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

    list_converted, abs_errors, rel_errors, total_time, t0, t1, t2, t3, file_size = _lets_convert(test_loader)
    for i in range(len(abs_errors)):
        if i == 0:
            abs_array = abs_errors[i]
            rel_array = rel_errors[i]
        else:
            np.append(abs_array, abs_errors[i])
            np.append(rel_array, rel_errors[i])
            
    abs_list = []
    rel_list = []
    model_list = []
    batch_list = []
    summary_list = ['Modelsize:{}, Conversion: {}, Loading: {}, t1: {}, conversion time: {}, onnx runtime: {}, onnx filesize: {}'.format(size0, total_time, t0, t1, t2, t3, file_size)]
    for i in range(len(abs_array)):
        abs_list.append(abs_array[i][0])
        rel = rel_array[i][0]
        if rel == np.inf or rel == -np.inf:
            rel = 0.0
        rel_list.append(rel)
        batch_list.append(batch_size)
        model_list.append(model_name)
        if i >= len(summary_list):
            summary_list.append('')
    print(len(summary_list), len(rel_list))
    data = pd.DataFrame({'model':model_list,'batch_size': batch_list, 'abs_errors':abs_list, 'rel_errors':rel_list, 'summary': summary_list})
    data.to_csv('./data/errors/torch2/tf_errors_{}_{}.csv'.format(model_name, batch_size))

converting for batch:  0


RuntimeError: Unsupported: ONNX export of operator adaptive_avg_pool2d, since output size is not factor of input size. Please feel free to request support or submit a pull request on PyTorch GitHub.

In [30]:
#np.testing.assert_equal(to_numpy(torch_out), ort_outs[0])